In [4]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletion

ModuleNotFoundError: No module named 'autogen_ext'

In [2]:
load_dotenv()

True

In [3]:
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY = os.getenv("OPENAI_API_KEY")
API_VERSION = "2024-05-01-preview"
AZURE_DEPLOYMENT_NAME = os.getenv("DEPLOYMENT_NAME")

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=API_VERSION,
    model=AZURE_DEPLOYMENT_NAME
)   
